#**TitleTronLLM: From abstracts to elegant titles**

 **Title Generator for Research Papers**

 This will take as input an abstract of your research paper and generate an apt title based on it. Mistral-7B, An open-source LLM is fined tuned on a dataset consisting of research paper abstracts and their titles which is extracted by me using NASA-ADS API for this purpose.



**The aim of this notebook is merely to fine-tune Mistral 7B model, In order to later use it for inference.**

In [ ]:
# Making sure dependencies work in the desired way

%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [ ]:
%pip install datasets==2.16.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

2024-03-13 15:57:25.953894: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 15:57:25.954015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 15:57:26.104299: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("huggingface")
secret_wandb = user_secrets.get_secret("weightsandbiases")

In [ ]:
# For enabling the huggingface functionality

!huggingface-cli login --token $secret_hf

'!huggingface-cli login --token $secret_hf'

In [ ]:
# Monitering the training with Weights and Biases

wandb.login(key = secret_wandb)
run = wandb.init(
    project='mistral7b',
    job_type="training",
    anonymous="allow"
)


In [ ]:
# Model directories

base_model = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
new_model = "mistral_7b_finetuned"


In [ ]:
# Loading the dataset

import pandas as pd
df=pd.read_csv('/kaggle/input/ads-top70k/ADS-top70K.csv',usecols = ['abstract', 'title'])

# For this notebook we are limiting the samples to 13000 but will fine-tune on full dataset too later

df.drop_duplicates(inplace = True)
df.dropna(inplace  = True)
df=df.head(13000)


In [ ]:
# Getting the dataset in desired format

def formatting_prompts_func(example):
    output_texts = []
    for i in range(df.shape[0]):
        text = f"<s>[INST] Craft an intelligent, clear, insightful, and succinct one-line title for the research paper, drawing inspiration from the abstract provided. \n {example.iloc[i,0]} [/INST] {example.iloc[i,1]} </s>"
        output_texts.append(text)
    return output_texts


In [ ]:
# Applying the above function on our dataframe

dataset = formatting_prompts_func(df)

In [ ]:
# A sample from dataset

dataset[1]

'<s>[INST] Craft an intelligent, clear, insightful, and succinct one-line title for the research paper, drawing inspiration from the abstract provided. \n we present yolo, a new approach to object detection. prior work on object detection repurposes classifiers to perform detection. instead, we frame object detection as a regression problem to spatially separated bounding boxes and associated class probabilities. a single neural network predicts bounding boxes and class probabilities directly from full images in one evaluation. since the whole detection pipeline is a single network, it can be optimized end-to-end directly on detection performance. our unified architecture is extremely fast. our base yolo model processes images in real-time at 45 frames per second. a smaller version of the network, fast yolo, processes an astounding 155 frames per second while still achieving double the map of other real-time detectors. compared to state-of-the-art detection systems, yolo makes more loc

In [ ]:
# Defining the 4 Bit Quantization configuration

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)


In [ ]:
# Loading the quantized model in memory

model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# Loading the tokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token


(True, True)

In [ ]:
# Setting the LoRA configuration

lora_config = LoraConfig(
    r=64,
    lora_dropout=0.1,
    lora_alpha=32,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)


In [ ]:
# Attach LoRa Adapters in the Layers

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_config
)
model = get_peft_model(model, peft_config)


In [ ]:
# Defining the training arguments

training_arguments = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=3,
    logging_steps=10,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    gradient_checkpointing_kwargs={'use_reentrant':True},
    report_to="wandb"
)


In [ ]:
# Getting the dataset to be compatible with Mistral

from datasets import Dataset
dataset2 = Dataset.from_dict({"text": dataset})

In [ ]:
# A sample from transformed dataset

dataset2["text"][1]

'<s>[INST] Craft an intelligent, clear, insightful, and succinct one-line title for the research paper, drawing inspiration from the abstract provided. \n we present yolo, a new approach to object detection. prior work on object detection repurposes classifiers to perform detection. instead, we frame object detection as a regression problem to spatially separated bounding boxes and associated class probabilities. a single neural network predicts bounding boxes and class probabilities directly from full images in one evaluation. since the whole detection pipeline is a single network, it can be optimized end-to-end directly on detection performance. our unified architecture is extremely fast. our base yolo model processes images in real-time at 45 frames per second. a smaller version of the network, fast yolo, processes an astounding 155 frames per second while still achieving double the map of other real-time detectors. compared to state-of-the-art detection systems, yolo makes more loc

In [ ]:
# Setting SFT Trainer parameters

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset2,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)


/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/13000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.963800
20,1.931600
30,1.867100
40,1.852600
50,1.741200
60,1.728200
70,1.778400
80,1.787400
90,1.820000
100,1.774700


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/mistral/pytorch/7b-v0.1-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/mistral/pytorch/7b-v0.1-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: t

TrainOutput(global_step=1083, training_loss=1.7261364871665272, metrics={'train_runtime': 30983.6961, 'train_samples_per_second': 0.42, 'train_steps_per_second': 0.035, 'total_flos': 2.016554844460155e+17, 'train_loss': 1.7261364871665272, 'epoch': 1.0})

In [ ]:
# Save the fine-tuned model

trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()


PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_pr

In [ ]:
# Push the model to Huggingface

trainer.model.push_to_hub("username/directoryname")


'trainer.model.push_to_hub("username/directoryname")'

In [ ]:
# Testing the fine-tuned model on an abstract input that is out of training data

prompt = "The recent GPT-4 has demonstrated extraordinary multi-modal abilities, such as directly generating websites from handwritten text and identifying humorous elements within images. These features are rarely observed in previous vision-language models. However, the technical details behind GPT-4 continue to remain undisclosed. We believe that the enhanced multi-modal generation capabilities of GPT-4 stem from the utilization of sophisticated large language models (LLM). To examine this phenomenon, we present MiniGPT-4, which aligns a frozen visual encoder with a frozen advanced LLM, Vicuna, using one projection layer. Our work, for the first time, uncovers that properly aligning the visual features with an advanced large language model can possess numerous advanced multi-modal abilities demonstrated by GPT-4, such as detailed image description generation and website creation from hand-drawn drafts. Furthermore, we also observe other emerging capabilities in MiniGPT-4, including writing stories and poems inspired by given images, teaching users how to cook based on food photos, and so on. In our experiment, we found that the model trained on short image caption pairs could produce unnatural language outputs (e.g., repetition and fragmentation). To address this problem, we curate a detailed image description dataset in the second stage to finetune the model, which consequently improves the model's generation reliability and overall usability."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=400)
result = pipe(f"<s>[INST] {prompt} [/INST]")


The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'P

In [ ]:
# Seeing the generated title

print(result)

[{'generated_text': "<s>[INST] The recent GPT-4 has demonstrated extraordinary multi-modal abilities, such as directly generating websites from handwritten text and identifying humorous elements within images. These features are rarely observed in previous vision-language models. However, the technical details behind GPT-4 continue to remain undisclosed. We believe that the enhanced multi-modal generation capabilities of GPT-4 stem from the utilization of sophisticated large language models (LLM). To examine this phenomenon, we present MiniGPT-4, which aligns a frozen visual encoder with a frozen advanced LLM, Vicuna, using one projection layer. Our work, for the first time, uncovers that properly aligning the visual features with an advanced large language model can possess numerous advanced multi-modal abilities demonstrated by GPT-4, such as detailed image description generation and website creation from hand-drawn drafts. Furthermore, we also observe other emerging capabilities in 